In [ ]:
## Reference: https://www.youtube.com/watch?v=xekqR10lQNo
## For personal study purposes only


## subclassification 

* stratify propensity scores into bins
* compare the treatment and control units within the bins to get the treatment effect estimation

In [1]:
from dowhy import datasets

import pandas as pd
import numpy as np

from causalinference import CausalModel

In [2]:
data = datasets.linear_dataset(
    beta=10,
    num_common_causes=4,
    num_samples=10_000,
    treatment_is_binary=True,
    outcome_is_binary=False,
)

df = data['df']
df = df.rename({'v0': 'treatment', 'y': 'outcome'}, axis=1)
df['treatment'] = df['treatment'].astype(int)

In [3]:
df.head()

,W0,W1,W2,W3,treatment,outcome
0,-2.410690,-1.941895,1.322482,2.095104,1,7.096365
1,-0.584717,-2.607352,-0.321803,-0.154031,0,-5.470646
2,-1.024231,-1.063797,-0.535115,-0.758109,0,-9.323665
3,0.846595,0.463510,0.583025,0.344339,1,17.306644
4,0.173637,-0.492804,0.079734,-0.264677,1,10.256141


In [4]:
causal = CausalModel(
    Y=df['outcome'].values,
    D=df['treatment'].values,
    X=df[['W0', 'W1', 'W2', 'W3']].values,
)

print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=6403)        Treated (N_t=3597)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -5.627        4.886       14.390        4.256       20.017

                      Controls (N_c=6403)        Treated (N_t=3597)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.597        0.926        0.279        0.917        0.950
             X1       -0.995        0.983       -0.704        0.985        0.296
             X2       -0.270        0.909        0.630        0.891        1.000
             X3       -0.513        0.896        0.437        0.851        1.087



## propensity score estimation

In [6]:
causal.est_propensity_s()

print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      0.049      0.061      0.791      0.429     -0.072      0.169
            X3      4.392      0.119     36.828      0.000      4.158      4.626
            X2      4.108      0.112     36.789      0.000      3.889      4.326
            X0      3.882      0.106     36.532      0.000      3.674      4.090
            X1      1.296      0.056     22.955      0.000      1.185      1.406



## subclassification matching by propensity score stratification

In [9]:
causal.stratify_s()

print(causal.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.000     0.059      4967        34     0.006     0.030    13.597
         2     0.059     0.165       558        67     0.100     0.106    10.455
         3     0.165     0.267       258        55     0.210     0.214     9.450
         4     0.267     0.410       194       118     0.334     0.342     9.755
         5     0.411     0.571       158       155     0.486     0.496     9.964
         6     0.571     0.722       114       198     0.645     0.653    10.275
         7     0.723     0.830        75       238     0.778     0.779    10.451
         8     0.831     0.852        19        59     0.841     0.842    10.835
         9     0.852     0.874         5        73     0.869     0.864    12.061
   

## subclassification treatment effect estimation

In [11]:
causal.est_via_blocking()

print(causal.estimates)


Treatment Effect Estimates: Blocking

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     10.000      0.001   9945.576      0.000      9.998     10.002
           ATC     10.000      0.001   6856.463      0.000      9.997     10.003
           ATT     10.000      0.001  10441.276      0.000      9.998     10.002



/Users/kai/anaconda3/lib/python3.10/site-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]
